<a href="https://colab.research.google.com/github/Ndeye-Ngone/GDA_Live_coding_FML23/blob/main/Ndeye_Ngone_GUEYE_Live_coding_GDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [2]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y=generate_data()# get data
print(x.shape, y.shape)

(1000, 3) (1000,)


In [3]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
  
  n=x.shape[0]

  suffle=np.random.permutation(n) 
  x_suffle=x[suffle]
  y_suffle=y[suffle]

  x_train=x_suffle[:int(train_size*n)]

  y_train=y_suffle[:int(train_size*n)]

  x_test=x_suffle[int(train_size*n):]

  y_test=y_suffle[int(train_size*n):]

  return x_train,x_test,y_train,y_test

In [4]:
X_train, X_test, y_train, y_test=split_data(x,y, train_size= 0.8) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [5]:
def covariance(x,mu):

  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.

  n,d=x.shape

  cov=np.zeros((d,d))

  for i in range(d):

    for j in range(d):

      cov[i][j]=( np.sum( (x[:,i]-mu[i]) * (x[:,j]-mu[j]) ) )/(n-1)

  return cov


def mu_(x,y,k):
    n,d=x.shape
    indicator=0
    x_sum=np.zeros(d)
    for i in range(n):
      if y[i]==k:
        indicator+=1
        x_sum+=x[i]
    return x_sum/indicator


def phi_(y,k):
    n=y.shape[0]
    indicator=0
    for i in range(n):
       if y[i]==k:
        indicator+=1
    return indicator/n


def gaussien_density(x,mu,sigma):
  #x=x.reshape(1,-x.shape[0])
  #mu=mu.reshape(1,-mu.shape[0])
  #d=x.shape[1]

  d=x.shape[0]

  const=(2*np.pi)**(d/2)

  det=np.linalg.det(sigma)
  
  x_mean=x-mu

  X=( x_mean.T@(np.linalg.inv(sigma)))@(x_mean)

  return np.exp(-X/2)/((const)*(det**(1/2)))


def bernoulli(phi,k):
  return (phi**k)*((1-phi)**(1-k))





In [6]:
x1=np.array([[2,3],[1,2],[2,4],[0,2]])

In [7]:
y1=np.array([[1],[0],[1],[0]])

In [8]:
mu_x=mu_(x1,y1,0)
mu_x

array([0.5, 2. ])

In [9]:
mu_y=mu_(x1,y1,1)
mu_y

array([2. , 3.5])

In [10]:
a=np.array([[2,1],[4,5],[1,2],[3,4],[7,6]])
b=np.array([[1],[1],[1],[1],[1]])
mu=mu_(a,b,1)
mu

array([3.4, 3.6])

In [11]:
np.cov(a,rowvar=0)

array([[5.3, 4.2],
       [4.2, 4.3]])

In [12]:
covariance(a,a.mean(0))

array([[5.3, 4.2],
       [4.2, 4.3]])

In [13]:
np.cov(x,rowvar=0)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [14]:
covariance(x,x.mean(0))

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [15]:
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu=None
    self.sigma=None
    self.phi=None

    
  def fit(self,x,y):
    k=2 # Number of class.
    d=x.shape[1] # input dim
    m= x.shape[0] # Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu=np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma=np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros(k)# k-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.

    for lab in range(k):

      bool_=(lab==y)

      self.phi[lab]=phi_(y,lab)

      self.mu[lab]=mu_(x,y,lab)

      self.sigma[lab]=covariance(x[bool_],self.mu[lab])


  def predict_proba(self,x):
    # reshape or flatt x.
   # x= ...
    d=x.shape[1]
    k_class= self.mu.shape[0]# Number of classes we have in our case it's k = 2
    
    ## START THE LEARNING: estimate mu, phi and sigma.
    proba=np.zeros((x.shape[0],k_class))

    for lab in range(k_class):

      for i in range(x.shape[0]):
        
        ###proba[i][lab]=gaussien_density(x[i],self.mu[lab],self.sigma[lab])*bernoulli(self.phi[lab],lab)

        proba[i][lab]=gaussien_density(x[i],self.mu[lab],self.sigma[lab])*self.phi[lab]
    
    return proba


  def predict(self,x):

    proba=self.predict_proba(x)

    y_pred=np.argmax(proba,axis=1)

    #self.y_pred=y_pred

    return y_pred
  
  
  def accuracy(self, y, ypreds):
    return np.mean(y==ypreds)*100



In [16]:
model= GDA()
model.fit(X_train,y_train)

In [17]:
model.phi

array([0.49125, 0.50875])

In [18]:
model.mu

array([[ 0.99189294,  1.13508193,  1.00901412],
       [-1.01714153,  0.95847114, -0.96403797]])

In [19]:
model.sigma

array([[[ 0.89094492, -0.38653686, -0.06587326],
        [-0.38653686,  1.69225741,  0.09578715],
        [-0.06587326,  0.09578715,  0.03734381]],

       [[ 0.80965789,  0.33003743,  0.14684351],
        [ 0.33003743,  0.33600361, -0.07072688],
        [ 0.14684351, -0.07072688,  1.61965731]]])

In [20]:
#gaussien_density(X_train[0],model.mu[0],model.sigma[0])

In [21]:
yproba= model.predict_proba(X_test)
yproba

array([[1.02577013e-022, 5.38490414e-002],
       [2.99356837e-030, 3.93062783e-002],
       [7.64525774e-002, 6.75394423e-004],
       [2.37928498e-002, 2.99153945e-006],
       [6.27117691e-003, 4.61025553e-021],
       [7.53831435e-063, 4.06263681e-002],
       [9.84194642e-091, 1.03803293e-002],
       [1.21473709e-012, 3.61465175e-002],
       [7.76599002e-014, 2.98262221e-002],
       [2.28376415e-002, 9.31778722e-004],
       [6.92129222e-002, 4.41120143e-009],
       [6.04800826e-002, 3.01093364e-003],
       [1.12209383e-001, 2.73507494e-004],
       [1.07976316e-002, 1.76604714e-006],
       [6.37935327e-011, 1.05277157e-003],
       [6.66626873e-002, 8.50551389e-008],
       [4.56640476e-004, 4.46967371e-006],
       [3.85285537e-002, 1.15292649e-004],
       [1.17710031e-016, 3.70937204e-002],
       [2.52062500e-084, 1.68300139e-002],
       [1.16717334e-001, 6.09402203e-006],
       [1.17450978e-003, 1.65416170e-016],
       [1.42603445e-052, 6.26361196e-003],
       [9.8

In [22]:
ypreds= model.predict(X_test)
ypreds


array([1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0])

In [23]:
model.accuracy(y_test, ypreds)

97.5

In [31]:
class LogisticRegression:
  '''
  The goal of this class is to create a LogisticRegression class, 
  that we will use as our model to classify data point into a corresponding class
  '''
  def __init__(self,lr,n_epochs):
    self.lr = lr
    self.n_epochs = n_epochs
    self.train_losses = []
    self.w = None
    self.weight = []

  def add_ones(self, x):
    ##### WRITE YOUR CODE HERE #####
    one = np.ones((x.shape[0],1))
    return np.hstack((one,x))
    #### END CODE ####

  def sigmoid(self, x):
    return 1/(1+np.exp(-x@self.w))


  def cross_entropy(self, x, y_true):
    ##### WRITE YOUR CODE HERE #####
    y_pred = self.sigmoid(x)
    loss = -np.mean(y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    return loss
    #### END CODE ####
  
  def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
    ##### WRITE YOUR CODE HERE #####
    x= self.add_ones(x)
    proba = self.sigmoid(x)
    return proba
    #### END CODE ####

  def predict(self,x):
    ##### WRITE YOUR CODE HERE #####
    probas = self.predict_proba(x)
    output = [0 if p<0.5 else 1 for p in probas]#np.where(probas>=0.5, 1, 0)      #convert the probalities into 0 and 1 by using a treshold=0.5
    return output
    #### END CODE ####

  def fit(self,x,y):
    # Add ones to x
    x=self.add_ones(x)

    # reshape y if needed
    y=y.reshape(-1,1)

    # Initialize w to zeros vector >>> (x.shape[1])
    self.w=np.zeros((x.shape[1],1))

    for epoch in range(self.n_epochs):
      # make predictions
      ypred = self.sigmoid(x)

      #compute the gradient
      dl = (-1/x.shape[0])*(x.T@(y-ypred))

      #update rule
      self.w=self.w-self.lr*dl

      #Compute and append the training loss in a list
      loss = self.cross_entropy(x,y)
      self.train_losses.append(loss)

      if epoch%1000 == 0:
        print(f'loss for epoch {epoch}  : {loss}')

  def accuracy(self,y_true, y_pred):
    ##### WRITE YOUR CODE HERE #####
    acc = np.mean(y_true==y_pred)*100
    return acc
    #### END CODE ####

In [32]:
model_R = LogisticRegression(0.01,n_epochs=10000)
model_R.fit(X_train,y_train)


loss for epoch 0  : 0.6881971314839734
loss for epoch 1000  : 0.1955306716655516
loss for epoch 2000  : 0.1709045542993211
loss for epoch 3000  : 0.16134226142464725
loss for epoch 4000  : 0.15597953879312473
loss for epoch 5000  : 0.15242952021865422
loss for epoch 6000  : 0.14985663597163124
loss for epoch 7000  : 0.14788756596575856
loss for epoch 8000  : 0.14632684322132927
loss for epoch 9000  : 0.14505982103405585


In [33]:
ypred_train = model_R.predict(X_train)
acc = model_R.accuracy(y_train,ypred_train)
print(f"The training accuracy is: {acc}")
print(" ")

ypred_test = model_R.predict(X_test)
acc = model_R.accuracy(y_test,ypred_test)
print(f"The test accuracy is: {acc}")

The training accuracy is: 95.875
 
The test accuracy is: 95.5


In [24]:
#Load the iris dataset from sklearn
X, y = make_classification(n_features=2, n_redundant=0, 
                           n_informative=2, random_state=1, 
                           n_clusters_per_class=1)

In [25]:
def train_test_split(X,y):
  '''
  this function takes as input the sample X and the corresponding features y
  and output the training and test set
  '''
  np.random.seed(0) # To demonstrate that if we use the same seed value twice, we will get the same random number twice
  
  train_size = 0.8
  n = int(len(X)*train_size)
  indices = np.arange(len(X))
  np.random.shuffle(indices)
  train_idx = indices[: n]
  test_idx = indices[n:]
  X_train, y_train = X[train_idx], y[train_idx]
  X_test, y_test = X[test_idx], y[test_idx]

  return X_train, y_train, X_test, y_test

In [26]:
X_train_, y_train_, X_test_, y_test_ = train_test_split(X,y)
print(f" the training shape is: {X_train.shape}")
print(f" the test shape is: {X_test.shape}")

 the training shape is: (800, 3)
 the test shape is: (200, 3)


In [27]:
model_= GDA()
model_.fit(X_train_,y_train_)

In [28]:
yproba= model_.predict_proba(X_test_)
yproba

array([[1.11014876e-42, 6.14021076e-01],
       [1.95061563e-02, 6.04563783e-11],
       [7.10316808e-02, 5.44637293e-09],
       [1.54663108e-31, 5.66293667e-01],
       [8.88769082e-01, 1.33564646e-09],
       [6.17700729e-73, 6.62243049e-02],
       [3.88766721e-34, 7.23160294e-01],
       [1.12207855e+00, 3.90529721e-10],
       [5.60117123e-33, 2.87454331e-01],
       [8.72887431e-01, 4.19018690e-12],
       [2.36936400e-25, 2.12023748e-01],
       [4.06250020e-01, 1.17223917e-08],
       [1.14574859e-01, 7.26831181e-13],
       [1.41657666e-52, 4.50964006e-01],
       [9.27741002e-24, 1.65288507e-02],
       [4.04353498e-52, 2.89338450e-01],
       [8.11522272e-01, 1.05102747e-11],
       [5.89138365e-01, 1.36709905e-09],
       [1.27870736e-45, 6.90756661e-01],
       [1.49690862e-20, 1.20969949e-02]])

In [29]:
ypreds= model_.predict(X_test_)
ypreds


array([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1])

In [30]:
model_.accuracy(y_test_, ypreds)

100.0